In [2]:
from collections import namedtuple

input_model = namedtuple("input_model", ["time", "set_k"])
row = namedtuple("row", ["i", "time", "set_k", "rn", "rk", "pn", "pk", "r"])

In [11]:
t = [10, 13, 5, 7, 9, 14, 12, 10, 6, 11, 9]
b = [{}, {}, {1}, {1}, {2}, {2}, {4,5}, {3,4,5}, {6,7}, {6,7}, {8,9}]

In [15]:
t = [5, 8, 3, 6, 4, 1, 2, 6, 3, 9, 7]
b = [{}, {}, {}, {1}, {1}, {3}, {2,5,6}, {2,5,6}, {4,7}, {3}, {2,5,6,10}]

In [1]:
t = [15, 7, 5, 6, 8, 6, 10, 8, 9, 11, 10, 9]
b = [{}, {}, {1}, {2}, {2}, {3,4}, {3,4}, {5,6}, {5,6}, {5,6}, {2}, {7,8,9}]

In [37]:
models: dict[int, input_model] = dict()

In [38]:
for i in range(1, len(t) + 1):
    models[i] = input_model(t[i-1], b[i-1])

In [39]:
models

{1: input_model(time=5, set_k={}),
 2: input_model(time=8, set_k={}),
 3: input_model(time=3, set_k={}),
 4: input_model(time=6, set_k={1}),
 5: input_model(time=4, set_k={1}),
 6: input_model(time=1, set_k={3}),
 7: input_model(time=2, set_k={2, 5, 6}),
 8: input_model(time=6, set_k={2, 5, 6}),
 9: input_model(time=3, set_k={4, 7}),
 10: input_model(time=9, set_k={3}),
 11: input_model(time=7, set_k={2, 10, 5, 6})}

In [40]:
table: dict[int, row] = dict()

In [41]:
for number, model in models.items():
    time = model.time
    set_k = model.set_k
    if len(set_k) == 0:
        rn = 0
    else:
        rn = max([table[s].rk for s in set_k])
    rk = rn + model.time 
    table[number] = row(i=number, time=time, set_k=set_k, rn=rn, rk=rk, pn=None, pk=None, r=None)

In [42]:
table

{1: row(i=1, time=5, set_k={}, rn=0, rk=5, pn=None, pk=None, r=None),
 2: row(i=2, time=8, set_k={}, rn=0, rk=8, pn=None, pk=None, r=None),
 3: row(i=3, time=3, set_k={}, rn=0, rk=3, pn=None, pk=None, r=None),
 4: row(i=4, time=6, set_k={1}, rn=5, rk=11, pn=None, pk=None, r=None),
 5: row(i=5, time=4, set_k={1}, rn=5, rk=9, pn=None, pk=None, r=None),
 6: row(i=6, time=1, set_k={3}, rn=3, rk=4, pn=None, pk=None, r=None),
 7: row(i=7, time=2, set_k={2, 5, 6}, rn=9, rk=11, pn=None, pk=None, r=None),
 8: row(i=8, time=6, set_k={2, 5, 6}, rn=9, rk=15, pn=None, pk=None, r=None),
 9: row(i=9, time=3, set_k={4, 7}, rn=11, rk=14, pn=None, pk=None, r=None),
 10: row(i=10, time=9, set_k={3}, rn=3, rk=12, pn=None, pk=None, r=None),
 11: row(i=11, time=7, set_k={2, 10, 5, 6}, rn=12, rk=19, pn=None, pk=None, r=None)}

In [43]:
def search_next_numbers(work_idx: int) -> set[int]:
    """Поиск следующих номеров работ, которые зависят от данной."""
    next_nums = {idx for idx, model in models.items()
                 if work_idx in model.set_k and idx != work_idx}  # Проверка на цикличность
    for next_num in next_nums.copy():
        next_nums.update(search_next_numbers(next_num))
    next_nums.discard(work_idx)
    return next_nums

In [44]:
for number, model in list(models.items())[::-1]:
    visited = search_next_numbers(number)
    current_row = table[number]
    if visited:
        k = min(visited, key=lambda num: table[num].pn if table[num].pn is not None else 10000000000)
        pk = table[k].pn
    else:
        pk = current_row.rk
    pn = pk - current_row.time
    r = pn - current_row.rn
    assert r == pk - current_row.rk and r >= 0, print(f"{current_row}, r={r}, {pk - current_row.rk}")
    table[number] = table[number]._replace(pk=pk, pn=pn, r=r)

In [45]:
from prettytable import PrettyTable


def print_table(data: dict[int, row]):
    pretty_table = PrettyTable()

    pretty_table.field_names = ["i", "t(i)", "K(i)", "t(rn, i)", "t(rk, i)", "t(pn, i)", "t(pk, i)", "r(i)"]

    for line in data.values():
        pretty_set = line.set_k if len(line.set_k) > 0 else "{}"
        pretty_table.add_row([line.i, line.time, pretty_set, line.rn, line.rk, line.pn, line.pk, line.r])

    print(pretty_table)


print_table(table)

+----+------+---------------+----------+----------+----------+----------+------+
| i  | t(i) |      K(i)     | t(rn, i) | t(rk, i) | t(pn, i) | t(pk, i) | r(i) |
+----+------+---------------+----------+----------+----------+----------+------+
| 1  |  5   |       {}      |    0     |    5     |    0     |    5     |  0   |
| 2  |  8   |       {}      |    0     |    8     |    1     |    9     |  1   |
| 3  |  3   |       {}      |    0     |    3     |    0     |    3     |  0   |
| 4  |  6   |      {1}      |    5     |    11    |    5     |    11    |  0   |
| 5  |  4   |      {1}      |    5     |    9     |    5     |    9     |  0   |
| 6  |  1   |      {3}      |    3     |    4     |    8     |    9     |  5   |
| 7  |  2   |   {2, 5, 6}   |    9     |    11    |    9     |    11    |  0   |
| 8  |  6   |   {2, 5, 6}   |    9     |    15    |    9     |    15    |  0   |
| 9  |  3   |     {4, 7}    |    11    |    14    |    11    |    14    |  0   |
| 10 |  9   |      {3}      